https://stackoverflow.com/questions/892199/detect-record-audio-in-python

http://people.csail.mit.edu/hubert/pyaudio/

http://people.csail.mit.edu/hubert/pyaudio/#record-example

https://github.com/jeysonmc/python-google-speech-scripts/blob/master/stt_google.py

In [ ]:
import pyaudio
import wave
import time
import tensorflow as tf
import label_wav
import numpy as np
import matplotlib.pyplot as plt
import stt_google
%matplotlib inline

In [ ]:
def record(WAVE_OUTPUT_FILENAME, count_down=True):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    #RATE = 44100
    RATE=8000 # 256Kbps
    RECORD_SECONDS = 1

    if(count_down):
        N = 3
        while(N > 0):
            print(N)
            time.sleep(1)
            N -= 1
        print("* recording, please speak")

    frames = []
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    for i in range(0, round(RATE/ CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    stream.stop_stream()
    stream.close()
    p.terminate()

    if(count_down):
        print("* done recording")

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

def play(sound):
    CHUNK = 1024
    wf = wave.open(sound, 'rb')

    p = pyaudio.PyAudio()

    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    data = wf.readframes(CHUNK)

    while data != b'':
        stream.write(data)
        data = wf.readframes(CHUNK)

    stream.stop_stream()
    stream.close()

    p.terminate()

def RunSysCmd(cmd):
    import subprocess
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    print(output.decode('utf-8'))

class flags():
    def __init__(self, wav, labels='Pretrained_models/labels.txt', graph='Pretrained_models/DNN/DNN_S.pb',
                 input_name='wav_data:0', output_name='labels_softmax:0'):
        self.wav = wav
        self.labels = labels
        self.graph = graph
        self.input_name = input_name
        self.output_name = output_name
        with open(labels) as f:
            how_many_labels = len(f.readlines())
        self.how_many_labels = how_many_labels

In [ ]:
stt_google.RATE = 8000 # 256Kbps
stt_google.audio_int()

In [ ]:
stt_google.listen_for_speech(3000, 1, 'input.wav')
play('input.wav')
label_wav.FLAGS = flags('input.wav', graph='Pretrained_models/DNN/DNN_L.pb')
predictions = label_wav.main('kws')
label_wav.show_result(predictions)